In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/metadata_clean.csv')
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995


Pairwise similarity
vDocuments have to vectorised before a similarity between them can be ascertained.
Every document is depicted as a series of n numbers, where each number represents a dimension and n is the size of the vocabulary of the document put together.
# Popular vectorisers 
* CountVectorizer
* TF-IDFVectorizer

# Count Vectorizer
Vocabulary = all unique words
Remove all stop words
dimensionsion is what is left
Vector made from the number of times each vocab vord is repeated in sentence

# TF-IDF Vectorizer 
Termed frequency Inverse Document Frequency
Proportional to the number of times the term appear in a doc
Inversely prop to the number of docs the term appear in


In [2]:
orig_df = pd.read_csv('../data/movies_metadata.csv', low_memory = False)

In [3]:
df['overview'],df['id'] = orig_df['overview'], orig_df['id']
df.head()


,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [4]:
df.shape

(45466, 8)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df['overview'] = df['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape

(45466, 75827)

In [ ]:
print(tfidf_matrix)


  (0, 38088)	0.10739705953465474
  (0, 73468)	0.4809827114790238
  (0, 3159)	0.4117836571172595
  (0, 67874)	0.14878284660693247
  (0, 39012)	0.0871868917895906
  (0, 28729)	0.13311522181618415
  (0, 56872)	0.11124851086523602
  (0, 7491)	0.12380553184830105
  (0, 9087)	0.10635375129287979
  (0, 9874)	0.502803868613561
  (0, 38693)	0.2062792468281062
  (0, 58571)	0.11355918868736861
  (0, 1847)	0.140911774178889
  (0, 39423)	0.11907123344715954
  (0, 50914)	0.09190797940163037
  (0, 29238)	0.10093917370354447
  (0, 51108)	0.1343481728311918
  (0, 12490)	0.12544427954397822
  (0, 59519)	0.1300801610445509
  (0, 48558)	0.10339358185033236
  (0, 19641)	0.13281884272823927
  (0, 21887)	0.10438761058719499
  (0, 38030)	0.10142919482788752
  (0, 4388)	0.14748820342184052
  (0, 17764)	0.1348314953863925
  :	:
  (45464, 63811)	0.12338183251109078
  (45464, 19945)	0.13612977846000052
  (45464, 15394)	0.14870056078358065
  (45464, 20273)	0.16775054113971025
  (45465, 74030)	0.10752037301100263
 

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index, index = df['title']).drop_duplicates()
print(indices)

In [ ]:
def content_recommender(title,cosine_sim=cosine_sim, df=df, indices=indices):
    idx= indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [ ]:
content_recommender('The Lion King')

In [ ]:
cred_df = pd.read_csv('../data/credits.csv')
cred_df.head()

In [ ]:
key_df = pd.read_csv('../data/keywords.csv')
key_df.head()

Id Clean UP

In [ ]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
df['id'] = df['id'].apply(clean_ids)

In [ ]:
df = df[df['id'].notnull()]

In [ ]:
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

# Merging


In [ ]:
df = df.merge(cred_df, on = 'id')
df = df.merge(key_df, on = 'id')

In [ ]:
df.head()

# Changing the JSON

In [ ]:
#Convert stringified to native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [ ]:
df.head()


In [ ]:
df['crew'].iloc[0]


In [ ]:
df.iloc[0]['crew'][0]

# Director extractor


In [ ]:
def get_director(x) :
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
        return np.nan

In [ ]:
df['director']= df['crew'].apply(get_director)
df['director'].head()

In [ ]:
df.head()

# Cast Crew Extractor


In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []
            

In [ ]:
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [ ]:
df.head()

# Sanitizer

In [ ]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [ ]:
df.head()

# Soup


In [ ]:
def create_soup(x):
    return ' '.join(x['keywords'])+' ' +' '.join(x['cast']) + ' ' + x['director']+' ' +' '.join(x['genres'])

In [ ]:
df['soup'] = df.apply(create_soup, axis =1)

In [ ]:
df.iloc[0]['soup']

## Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
print(cosine_sim2)

In [ ]:
df =df.reset_index()
indices2 =pd.Series(df.index, index=df['title'])

In [ ]:
content_recommender('The Lion King',cosin_sim2,df,indices2)